# TP 2 - EXTRACTION DE L'INFORMATION : VIDEO

Nour Elhouda KIRED -
Valentine ROSSI -
Théo VEDIS

In [1]:
import matplotlib.pyplot as plt
import numpy as np
!pip install tqdm
# from tqdm.notebook import tqdm # permet l'affiche d'une barre de chargement
# Si ca ne marche pas
from tqdm import tqdm

In [2]:
# Récupération des fonctions du TP1
def quantificationV2(img, nb=2, nbbit=8):
    imgQuantif = []
    for i in range(len(img)):
        imgQuantif.append([])
        for j in range(len(img[i])):
            imgQuantif[i].append(0)
            for k in range(len(img[i][j])):
                imgQuantif[i][j] += (img[i][j][k] >> (nbbit - nb)) << (nb*(2-k))

    return np.array(imgQuantif)

def histogramme(img, nb=6):
    """
        Prend une image quantifiée en entrée!
        nb = nombre de bits total pour les canaux
    """
    histo = np.zeros(2**nb, dtype=int)
    for line in img:
        for i in line:
            histo[i] += 1
    return histo

def indexation(images, nb=2, nbCanaux=3):
    histos = {}
    for key, img in tqdm(images.items()):
        histos[key] = (histogramme(quantificationV2(img, nb=nb), nb=nb*nbCanaux))
    return histos

def distanceHisto(h1, h2):
    """
        Renvoie la distance de Manhattan entre 2 histogrammes
    """
    val = 0
    for i in range(len(h1)):
        val += min(h1[i], h2[i])
    return val

def dstIntersect(histos, fct=distanceHisto):
    """
        Renvoie la matrice des distances entre 2 histogrammes de type intersection d'histogrammes
    """
    dst = {}
    for key1, histo1  in histos.items():
        dst[key1] = {}
        for key2, histo2 in histos.items():
            if key1 != key2:
                dst[key1][key2] = fct(histos[key1], histos[key2])
            else:
                dst[key1][key2] = 0
    return dst

In [3]:
# Lecture des images
images_paths = ["./Images_Groland/" + f for f in os.listdir("./Images_Groland/") if f[-4:] == ".jpg"]
images = {}
for path in images_paths:
    images[path[len("./Images_Groland/"):-4]] = plt.imread(path)


### Question 1-a

In [4]:
#  Dictionnaire contenant les histogrammes des images quantifiées sur 2 bits de poids fort
histos = indexation(images, 2)

100%|██████████| 795/795 [06:10<00:00,  2.15it/s]


### Question 1-b,c

In [5]:
def distManhattan(h1, h2):
    """
        Calcul la distance de manhattan entre 2 histogrammes
    """
    return np.sum(np.abs(h1-h2))

def dst2a2(histos):
    """
        Entrée <== Le dictionnaire contenant les histogrammes de chaque image
        Sortie ==> Le dictionnaire contenant les distances entre un histogramme et l'histogramme suivant
    """
    dst = {}
    for i in range(len(histos) - 1):
        dst[list(histos.keys())[i]] = distManhattan(histos[list(histos.keys())[i]], histos[list(histos.keys())[i+1]])
    return dst

dst = dst2a2(histos)

# On fixe le nombre de plans à 7 (déduit en regardant le film).
nbPlan = 7
seuil = sum(sorted(dst.values(), key=lambda x: -x)[nbPlan-1:nbPlan+1])/2
# Moyenne entre la 7 eme et la 8eme plus grosse valeur
print(seuil)


16789.0


### Question 1-d

In [6]:
groupes = []
deb = 0
for i in range(len(dst)):
    if dst[list(dst.keys())[i]] > seuil:
        groupes.append((list(dst.keys())[deb], list(dst.keys())[i]))
        deb = i+1

groupes.append((list(dst.keys())[deb],list(dst.keys())[-1]))
# Affichage des Images encadrant les plans
display(groupes)


imgMed = []
for a, b in groupes:
    name = str((int(a[8:]) + int(b[8:])) // 2)
    while len(name) < 3:
        name = '0' + name
    imgMed.append('Groland ' + name)

# Affichage des images médianes de chaque plan
display(imgMed)

[('Groland 001', 'Groland 115'),
 ('Groland 116', 'Groland 232'),
 ('Groland 233', 'Groland 402'),
 ('Groland 403', 'Groland 463'),
 ('Groland 464', 'Groland 612'),
 ('Groland 613', 'Groland 670'),
 ('Groland 671', 'Groland 710'),
 ('Groland 711', 'Groland 794')]

['Groland 058',
 'Groland 174',
 'Groland 317',
 'Groland 433',
 'Groland 538',
 'Groland 641',
 'Groland 690',
 'Groland 752']

### Question 2

In [7]:
dstMed = {}
for i in imgMed:
    dstMed[i] = {}
    for j in imgMed:
        if i != j:
            dstMed[i][j] = distManhattan(histos[i], histos[j])

# dstMad => Distance entre les images médianes de chaque plan

# print(dstMed)
couples = []
seuil = 10000
for i, distances in dstMed.items():
    for j, d in distances.items():
        if d < seuil:
            print(i, "|", j)
            couples.append((i, j))

sequences = []
# couples.insert(2, ("Groland 538", "Groland 431")) # Test dans un cas où les couples sont disjoints
for i in couples:
    isAdd = False
    fusion = []
    for sq, j in zip(sequences, range(len(sequences))):
        if (i[0] in sq or i[1] in sq):
            sq.add(i[0])
            sq.add(i[1])
            fusion.append((j, sq))
            isAdd = True
    if not isAdd:
        sequences.append(set())
        sequences[-1].add(i[0])
        sequences[-1].add(i[1])
    if len(fusion) > 1:
        nSq = set()
        count = 0
        for j in fusion:
            nSq = nSq.union(j[1])
            sequences.pop(j[0] - count)
            count += 1
        sequences.append(nSq)

# Regroupement des plans selon 2 sequences:
print("\nNum des images médianes regroupées :", *sequences)

Groland 317 | Groland 752
Groland 433 | Groland 690
Groland 538 | Groland 690
Groland 690 | Groland 433
Groland 690 | Groland 538
Groland 752 | Groland 317

Num des images médianes regroupées : {'Groland 317', 'Groland 752'} {'Groland 538', 'Groland 690', 'Groland 433'}
